In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn import CrossEntropyLoss

In [ ]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

from scipy import sparse
import os
from os import path
from pathlib import Path
import pickle
import operator

SEED = 3
np.random.seed(SEED)

## Loading data

In [ ]:
ML_DIR = "Data"
export_dir = Path(os.getcwd())
files_path = Path(export_dir.parent, ML_DIR)

In [ ]:
data = pd.read_csv(Path(files_path, "yahoo_music_subset.csv"), sep=",", engine="python",
                   names=["user_id_original","item_id_original", "rating"])

## Data statistics

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
# Users with one or two items in the interaction
user_ids = list(data.groupby('user_id_original')['item_id_original'].count()[lambda x: x < 3].index)

In [ ]:
# remove such users from the data set
filtered_data = data[~data['user_id_original'].isin(user_ids)]

In [ ]:
# Verification 
print(filtered_data.user_id_original.nunique()+len(user_ids) == data.user_id_original.nunique())

In [ ]:
data = filtered_data

In [ ]:
np.mean(data.groupby('user_id_original')['item_id_original'].count())

In [ ]:
# Convert the ratings to binary values (1 if rating exists, 0 otherwise)
data["rating"] = data["rating"].apply(lambda x: 1 if x > 0 else 0)


# Encode target values
data["user_id"] = LabelEncoder().fit_transform(data.user_id_original)
data["item_id"] = LabelEncoder().fit_transform(data.item_id_original)

# Get the number of users and items in the dataset
num_users = data.user_id.unique().shape[0]
num_items = data.item_id.unique().shape[0]

In [ ]:
num_users = data.user_id.unique().shape[0]
num_items = data.item_id.unique().shape[0]

In [ ]:
num_users

In [ ]:
num_items

## Data processing 

In [ ]:
user_group = data[["user_id","item_id"]].groupby(data.user_id)

users_data = pd.DataFrame(
    data={
        "user_id": list(user_group.groups.keys()),
        "item_ids": list(user_group.item_id.apply(list)),
    }    
)

In [ ]:
mlb = MultiLabelBinarizer()
user_one_hot = pd.DataFrame(mlb.fit_transform(users_data["item_ids"]),columns=mlb.classes_, index=users_data["item_ids"].index)

In [ ]:
user_one_hot["user_id"] = users_data["user_id"]

In [ ]:
user_one_hot

In [ ]:
user_one_hot.iloc[:,:-1]

In [ ]:
def get_y_values(numpy_arr):
    """
    Sample items from consumed items to predict (positive examples)
    """
    y_values = []
    users_arr = np.split(numpy_arr[:, 1], np.unique(numpy_arr[:, 0], return_index=True)[1][1:])
    for u in users_arr:
          y_values.append(int(np.random.choice(u[:-1],1,replace=False)))
    return y_values    

In [ ]:
y_indices = get_y_values(np.argwhere(user_one_hot.to_numpy()>0))

In [ ]:
len(y_indices)

In [ ]:
user_one_hot["y_positive"] = y_indices

In [ ]:
#Create representation of each item as a binary vector
items_values = pd.DataFrame(np.eye(num_items, dtype=int), columns=np.arange(num_items))

In [ ]:
items_values.to_csv(Path(export_dir, "items_values_Yahoo.csv"), index=False)

In [ ]:
items_values.index.values

In [ ]:
items_values_dict = {}
for i in range(items_values.shape[0]):
    items_values_dict[i] = items_values.iloc[i,:]

In [ ]:
file_path = 'items_values_dict_Yahoo.pkl'

# Open the file in write binary mode and use pickle.dump to save the dictionary
with open(Path(export_dir,file_path), 'wb') as f:
    pickle.dump(items_values_dict, f)

In [ ]:
data.user_id.unique()

## Split to training and testing

In [ ]:
random_state = 12
# The list of users IDs
users_indices = data.user_id.unique()

# Set the split ratio (80% for training, 20% for testing)
split_ratio = 0.8

# Calculate the split index
shuffled_users_ids = np.random.permutation(users_indices)
split_index = int(len(shuffled_users_ids) * split_ratio)

# Split the list of user IDs into training and testing sets
train_user_ids = shuffled_users_ids[:split_index]
test_user_ids = shuffled_users_ids[split_index:]


In [ ]:
train_user_ids

In [ ]:
# Split the user hot encoding matrix into training and testing sets based on the selected user IDs
train_data = user_one_hot.loc[train_user_ids]
test_data = user_one_hot.loc[test_user_ids]

In [ ]:
test_data

In [ ]:
training_data = data[data.user_id.isin(train_user_ids)]

# Group by item_id and aggregate user_id values into a list
item_group = training_data.groupby("item_id")

items_data = pd.DataFrame(
    data={
        "item_id": list(item_group.groups.keys()),
        "users_ids": list(item_group.user_id.apply(list)),
    }    
)

In [ ]:
items_data 

## Creating files for baseline calculation

In [ ]:
def jaccard_similarity(set1, set2):
    """
    Calculate the Jaccard similarity between two sets of genres
    """
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union

In [ ]:
def create_user_based_Jaccard_sim():
    """
    Calculate similarity between items based on users that consumed these items (Jaccard similarity)
    """
    item_similarities = {}
    for index1, row1 in items_data.iterrows():
        for index2, row2 in items_data.iterrows():
            if row1["item_id"]!= row2["item_id"]: #and item2>item1:
                similarity = jaccard_similarity(set(row1["users_ids"]), set(row2["users_ids"]))
            else:
                similarity = 1
            item_similarities[(row1["item_id"], row2["item_id"])] = similarity

    return item_similarities 

In [ ]:
user_similarities_Jaccard = create_user_based_Jaccard_sim()

In [ ]:
file_path = 'user_similarities_Jaccard_Yahoo.pkl'

# Open the file in write binary mode and use pickle.dump to save the dictionary
with open(Path(export_dir,file_path), 'wb') as f:
    pickle.dump(user_similarities_Jaccard, f)

In [ ]:
# Use MultiLabelBinarizer to encode the user IDs for each item into a one-hot matrix
mlb = MultiLabelBinarizer(classes=train_user_ids)  # Only include train_user_ids
item_one_hot = pd.DataFrame(
    mlb.fit_transform(items_data["users_ids"]),
    columns=mlb.classes_,
    index=items_data["item_id"]
)

In [ ]:
training_data.item_id.nunique()

In [ ]:
item_one_hot

In [ ]:
def item_user_based_cos_sim():
    """
    Item similarity based on cosine between user and item
    """
    return cosine_similarity(item_one_hot)

In [ ]:
cosine_items = item_user_based_cos_sim()

In [ ]:
cosine_items

In [ ]:
cosine_items_dict = {}

# Loop through the rows and columns of the ndarray and add each element to the dictionary
for i in range(cosine_items.shape[0]):
    for j in range(cosine_items.shape[1]):
        cosine_items_dict[(i, j)] = cosine_items[i][j]

In [ ]:
cosine_items_dict[(0,0)]

In [ ]:
file_path = 'cosine_items_Yahoo.pkl'

# Open the file in write binary mode and use pickle.dump to save the dictionary
with open(Path(export_dir, file_path), 'wb') as f:
    pickle.dump(cosine_items_dict, f)

In [ ]:
cosine_items = cosine_items_dict

### Add negative examples to training data & Calculate the popularity of each item in the training set

In [ ]:
popularity_dict = train_data.iloc[:,:-2].sum(axis=0).to_dict()

In [ ]:
file_path = 'pop_dict_Yahoo.pkl'

with open(Path(export_dir, file_path), 'wb') as f:
    pickle.dump(popularity_dict,f)  

In [ ]:
prob_dict = {}
for k, v in popularity_dict.items():
    prob_dict[k] = v / sum(popularity_dict.values())
print(len(prob_dict))

In [ ]:
file_path = 'prob_dict_Yahoo.pkl'

with open(Path(export_dir, file_path), 'wb') as f:
    pickle.dump(prob_dict,f)  

In [ ]:
sorted(prob_dict.items(), key=operator.itemgetter(1),reverse=True)[0]

In [ ]:
def get_negative_samples(numpy_arr,num):
    """
    Sample negative points 
    """
    negative_values = []
    users_arr = np.split(numpy_arr[:, 1], np.unique(numpy_arr[:, 0], return_index=True)[1][1:])
    for u in users_arr:
        items_from_dict_keys = [d for d in popularity_dict.keys() if d in u]
        sum_popularity= 0
        for it in items_from_dict_keys:
            sum_popularity += popularity_dict[it] 
        items_probs = [popularity_dict[d]/sum_popularity for d in items_from_dict_keys]
        negative_samples = np.random.choice(items_from_dict_keys, size=num, replace=False, p=items_probs)
        if(num == 1):
            negative_values.append(int(negative_samples))
        else:
            negative_values.append(negative_samples) 
                               
    return negative_values   

In [ ]:
y_negative = get_negative_samples(np.argwhere(train_data.iloc[:,:-2].to_numpy()==0), num=1)

In [ ]:
train_data["y_negative"] = y_negative

In [ ]:
train_data.head()

In [ ]:
train_data

### Positive and Negative examples merged together

In [ ]:
train_data_mixed = train_data.merge(train_data.loc[:,['user_id','y_positive','y_negative']].melt('user_id', value_name='y_values').replace({'y_positive': 1, 'y_negative': 0}), on="user_id").rename(columns={'variable': 'interaction'}).drop(['y_positive', 'y_negative'],axis=1)

In [ ]:
train_data_mixed.head()

In [ ]:
train_data_mixed.to_csv(Path(export_dir,'train_data_mixed_Yahoo.csv'), index=False)

In [ ]:
test_data.to_csv(Path(export_dir,"test_data_Yahoo.csv"), index=False)

In [ ]:
train_array = train_data_mixed.to_numpy()
test_array = test_data.to_numpy()

In [ ]:
import numpy as np

train_data =(train_array[train_array[:,-2]==0][:,:-3]).T
test_data = (test_array[:,:-2]).T

# Compute the IDF scores for each item in the train data
num_docs = train_data.shape[0]
idf = np.log(num_docs / (np.sum(train_data, axis=0) + 1))

# Compute the TF-IDF scores for each item in the train data
tfidf_matrix = np.zeros(train_data.shape)
for i in range(train_data.shape[1]):
    tf = train_data[:, i] / np.sum(train_data[:, i])
    tfidf_matrix[:, i] = tf * idf[i]  

tf_idf_items_dict = {(i, j): tfidf_matrix[i, j] for i in range(tfidf_matrix.shape[0]) for j in range(tfidf_matrix.shape[1])}

In [ ]:
file_path = 'tf_idf_items_Yahoo.pkl'

# Open the file in write binary mode and use pickle.dump to save the dictionary
with open(Path(export_dir, file_path), 'wb') as f:
    pickle.dump(tf_idf_items_dict, f)

## Prepare Models

In [ ]:
class MLP_G(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MLP_G, self).__init__()
        self.linear_x = nn.Linear(input_size, hidden_size, bias = False)
        self.linear_y = nn.Linear(input_size, hidden_size, bias = False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, user, item):
        user_representation = self.linear_x(user.float())
        item_representation = self.linear_y(item.float())
        dot_prod = torch.matmul(user_representation, item_representation.T)
        dot_sigmoid = self.sigmoid(dot_prod)
        
        return dot_sigmoid

In [ ]:
class Recommender_G(nn.Module):
    def __init__(self, num_items, hidden_size):
        super(Recommender_G, self).__init__()
        self.mlp = MLP_G(num_items, hidden_size).to(device)

    def forward(self, user_vector, item_vector):
        user_vector = user_vector.to(device)
        item_vector = item_vector.to(device)
        output = self.mlp(user_vector, item_vector)
        return output.to(device)

In [ ]:
def get_top_k(user_vector, original_user_vector, num_items, model, top_k):
    item_prob_dict = {}
    user_tensor = torch.Tensor(user_vector).to(device)
    item_tensor = torch.FloatTensor(items_array).to(device)
    output_model = [float(i) for i in model(user_tensor, item_tensor).cpu().detach().numpy()]
    
    original_user_vector = np.array(original_user_vector.cpu())
    neg = np.ones_like(original_user_vector)- original_user_vector
    output = neg*output_model
    for i in range(len(output)):
        item_prob_dict[i]=output[i]

    sorted_items_by_prob  = sorted(item_prob_dict.items(), key=lambda item: item[1],reverse=True)

    return dict(sorted_items_by_prob[0:top_k])

## Train data on these models

In [ ]:
# Train the model on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 20
print("num_users is ", num_users)
print("num_items is ", num_items)
recommender_model_g = Recommender_G(num_items, hidden_dim)
recommender_loss_g = nn.BCELoss()
recommender_optimizer_g = torch.optim.Adam(recommender_model_g.parameters(), lr=0.001)

In [79]:
train_losses = []
epochs = 10

for epoch in range(epochs):
    if epoch == 20:
        recommender_optimizer.lr = 0.0001
    train_loss = 0
    for i in range(train_array.shape[0]):
        item_id = train_array[i][-1]
        user_id = train_array[i][-3]
        item_vector = items_values_dict[item_id]
        user_vector = train_array[i][:-3]
        user_vector[item_id] = 0 
        interact = train_array[i][-2]
        
        user_tensor = torch.FloatTensor(user_vector).to(device)
        item_tensor = torch.FloatTensor(item_vector).to(device)
      
        interact_tensor = torch.FloatTensor([interact]).to(device)
        interact_tensor = torch.tensor(interact_tensor.item()).to(device)
        
        recommender_optimizer_g.zero_grad()
        recommender_output = (recommender_model_g(user_tensor, item_tensor)).to(device)
        rec_loss = recommender_loss_g(recommender_output, interact_tensor)
        
        train_loss+=rec_loss.item()
            
        rec_loss.backward()
        recommender_optimizer_g.step()
    train_losses.append(train_loss/train_array.shape[0])
    print(f"Epoch {epoch}, Train Loss {train_loss/train_array.shape[0]:.4f}")

KeyboardInterrupt: 

In [ ]:
torch.save(recommender_model_g.state_dict(), Path(export_dir,'recommender_model_yahoo.pt'))

### Recommender Freezing

In [ ]:
for param in recommender_model_g.parameters():
    param.requires_grad= False

In [ ]:
rec_model = recommender_model_g

## Save top k items in training and test set

In [ ]:
#Get users vectors to create topk
unique_indices = np.unique(train_array[:,-3], return_index=True, axis=0)[1]

# create a new array with only the unique users
train_unique_arr = train_array[unique_indices, :]
items_array = items_values.to_numpy()

In [ ]:
#Create top_k dictionary that contains user, item, score for the rec_model
#and is stored as (user_id, item_id, score) values
def create_topk_data(data_, rec_model, data_type = "train"):

    user_item_matrix = np.zeros((num_users, num_items))
    
    for i in range(data_.shape[0]):

        if(data_type == "train"):
            user_id = data_[i][-3]
            user_vector = data_[i][:-3]
        elif(data_type == "test"):
            user_id = data_[i][-2]
            user_vector = data_[i][:-2]
            
        user_tensor = torch.FloatTensor(user_vector).to(device)
            
        top_items = get_top_k(user_tensor, user_tensor, num_items, rec_model, num_items)
      
        for item_id in top_items.keys(): 
            user_item_matrix[user_id, item_id] = top_items[item_id]
        
        if(i % 100 == 0):
            print(i)
            
    return user_item_matrix


In [ ]:
topk_train = create_topk_data(train_unique_arr, rec_model, data_type="train")

In [ ]:
topk_test = create_topk_data(test_array, rec_model, data_type="test")

In [ ]:
file_path = 'topk_train_Yahoo.pkl'

# open the file in write-binary mode and save the array
with open(Path(export_dir, file_path), 'wb') as f:
    pickle.dump(topk_train, f)

In [ ]:
file_path = 'topk_test_Yahoo.pkl'

# open the file in write-binary mode and save the array
with open(Path(export_dir, file_path), 'wb') as f:
    pickle.dump(topk_test, f)